<a href="https://colab.research.google.com/github/k4kirin/mofsxml/blob/main/MOFsxML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MOFs x ML Project**


Written by Kirin and Ohm


**Import the necessary library**

In [ ]:
!git clone -b master https://github.com/k4kirin/mofsxml

Cloning into 'mofsxml'...
remote: Enumerating objects: 68621, done.
remote: Counting objects: 100% (68621/68621), done.
remote: Compressing objects: 100% (68618/68618), done.
remote: Total 68621 (delta 2), reused 68615 (delta 2), pack-reused 0
Receiving objects: 100% (68621/68621), 267.75 MiB | 19.60 MiB/s, done.
Resolving deltas: 100% (2/2), done.
Checking out files: 100% (68613/68613), done.


In [ ]:
!pip install ase
import pandas as pd
import numpy as np
import requests
import copy

from ase.io import read

     |████████████████████████████████| 2.2 MB 7.6 MB/s 


**Read the CSV (data) file from the github repository**

For a single file, it can be easily upload to the github repository on the website. However, if there are too many file like 70k cif file which we need for surface area calculation. There are more steps: add origin >> add * >> commit >> push.

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/k4kirin/mofsxml/main/train.csv',keep_default_na=True)

**Display the data from the previous step**

*see* the first 10 data in the dataset

In [ ]:
df.head(10)

,MOFname,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],functional_groups,metal_linker,organic_linker1,organic_linker2,topology,CO2/N2_selectivity,heat_adsorption_CO2_P0.15bar_T298K [kcal/mol],CO2_working_capacity [mL/g]
0,mof_unit_1,1116.667429,875.240600,0.00,0.07899,0.0607,COOH-OEt,3,4,11,pcu,22.864166,6.786041,105.284502
1,mof_unit_2,2769.503842,2211.697211,603.61,0.13794,0.1040,F-OMe,10,44,57,etb,33.616780,7.147286,101.224774
2,mof_unit_3,1089.818728,773.687960,788.50,0.14874,0.1262,OMe-COOH,2,22,24,pcu,19.263726,6.347967,118.987011
3,mof_unit_4,2205.198301,1304.638720,1441.53,0.21814,0.2220,H-SO3H,9,17,24,sra,25.701377,6.190085,187.626004
4,mof_unit_5,1137.800963,901.736120,0.00,0.07778,0.0591,NHMe-OH,2,1,22,pcu,30.001838,6.478063,79.210001
5,mof_unit_6,3954.659761,1543.027680,2430.55,0.37094,0.5725,Pr-NO2,9,7,23,sra,17.146541,5.398304,55.786959
6,mof_unit_7,3565.914939,1954.749656,1530.02,0.33337,0.3662,NH2,10,53,55,etb,18.363791,6.303857,111.690462
7,mof_unit_8,916.043907,639.111560,1685.58,0.28458,0.2456,Br,2,5,13,pcu,12.684804,5.234732,58.989774
8,mof_unit_9,2228.882339,1422.691820,1458.02,0.24277,0.2291,HCO,4,14,19,acs,37.040424,6.298964,135.587108
9,mof_unit_10,4275.421420,1606.196000,3276.11,0.41004,0.6573,NHMe-OH,3,13,29,pcu,10.999299,5.539327,59.739057


In [ ]:
#Preprocessing!!
df = pd.read_csv('https://raw.githubusercontent.com/k4kirin/mofsxml/main/train.csv',keep_default_na=True)

# remove rows when both void_fraction and void_volume == 0
before = len(df.index)
df = df.drop(df[(df.void_fraction == 0) & (df['void_volume [cm^3/g]'] == 0)].index)
print("removed",before - len(df.index),"rows")

# remove NaN rows
df.dropna()

# remove void_volume column
df = df.drop('void_volume [cm^3/g]',1)

# enumerate functional_groups & topology
col = ["functional_groups", "topology"]
for i in col:
    df[i] = df[i].astype("category").cat.codes

removed 2874 rows


In [ ]:
#TODO: fill in surface_area with values from that function if == 0
# default global numbers
E = 1e-3
PI = np.pi
angstrom = 1e-10

vdw_radii = {'H': 1.1, 'He': 1.4, 'Li': 1.82, 'Be': 1.53, 'B': 1.92, 'C': 1.7, 'N': 1.55, 
            'O': 1.52, 'F': 1.47, 'Ne': 1.54, 'Na': 2.27, 'Mg': 1.73, 'Al': 1.84, 'Si': 2.1, 
            'P': 1.8, 'S': 1.8, 'Cl': 1.75, 'Ar': 1.88, 'K': 2.75, 'Ca': 2.31, 'Sc': 2.15, 'Ti': 2.11, 
            'V': 2.07, 'Cr': 2.06, 'Mn': 2.05, 'Fe': 2.04, 'Co': 2.0, 'Ni': 1.97, 'Cu': 1.96, 'Zn': 2.01, 
            'Ga': 1.87, 'Ge': 2.11, 'As': 1.85, 'Se': 1.9, 'Br': 1.85, 'Kr': 2.02, 'Rb': 3.03, 'Sr': 2.49, 
            'Y': 2.32, 'Zr': 2.23, 'Nb': 2.18, 'Mo': 2.17, 'Tc': 2.16, 'Ru': 2.13, 'Rh': 2.1, 'Pd': 2.1, 
            'Ag': 2.11, 'Cd': 2.18, 'In': 1.93, 'Sn': 2.17, 'Sb': 2.06, 'Te': 2.06, 'I': 1.98, 'Xe': 2.16, 
            'Cs': 3.43, 'Ba': 2.68, 'La': 2.43, 'Ce': 2.42, 'Pr': 2.4, 'Nd': 2.39, 'Pm': 2.38, 'Sm': 2.36, 
            'Eu': 2.35, 'Gd': 2.34, 'Tb': 2.33, 'Dy': 2.31, 'Ho': 2.3, 'Er': 2.29, 'Tm': 2.27, 'Yb': 2.26, 
            'Lu': 2.24, 'Hf': 2.23, 'Ta': 2.22, 'W': 2.18, 'Re': 2.16, 'Os': 2.16, 'Ir': 2.13, 'Pt': 2.13,
            'Au': 2.14, 'Hg': 2.23, 'Tl': 1.96, 'Pb': 2.02, 'Bi': 2.07, 'Po': 1.97, 'At': 2.02, 'Rn': 2.2, 
            'Fr': 3.48, 'Ra': 2.83, 'Ac': 2.47, 'Th': 2.45, 'Pa': 2.43, 'U': 2.41, 'Np': 2.39, 'Pu': 2.43, 
            'Am': 2.44, 'Cm': 2.45, 'Bk': 2.44, 'Cf': 2.45, 'Es': 2.45, 'Fm': 2.45, 'Md': 2.46, 'No': 2.46, 
            'Lr': 2.46}
def get_box_parameters(cifname):
    """
    ดึงข้อมูลพารามิเตอร์ของ Unit Cell ของ MOF

    cifname : CIF file
    """
    content = open(cifname, 'r').readlines()
    for line in content:
        if "_cell_length_a" in line:
            a = float(line.split()[1])
        if "_cell_length_b" in line:
            b = float(line.split()[1])
        if "_cell_length_c" in line:
            c = float(line.split()[1])
        if "_cell_angle_alpha" in line:
            alpha_r = float(line.split()[1]) * PI / 180.0
        if "_cell_angle_beta" in line:
            beta_r = float(line.split()[1]) * PI / 180.0
        if "_cell_angle_gamma" in line:
            gamma_r = float(line.split()[1]) * PI / 180.0

    return a, b, c, alpha_r, beta_r, gamma_r
def get_box_matrix(cifname):
    """
    คำนวณ Unit Cell Matrix 

    cifname : CIF file
    """
    a, b, c, alpha_r, beta_r, gamma_r = get_box_parameters(cifname)

    a_x = a
    a_y = 0.0
    a_z = 0.0
    b_x = b * np.cos(gamma_r)
    b_y = b * np.sin(gamma_r)
    b_z = 0.0
    c_x = c * np.cos(beta_r)
    c_y = (b * c * np.cos(alpha_r) - b_x * c_x) / b_y
    c_z = np.sqrt(c**2 - c_x**2 - c_y**2)

    cell_matrix = np.matrix([
        [a_x, a_y, a_z], 
        [b_x, b_y, b_z], 
        [c_x, c_y, c_z]], 
        dtype=float)

    crs = np.cross([b_x, b_y, b_z], [c_x, c_y, c_z])
    inverse_matrix = cell_matrix.I
    unit_cell_volume = a_x * crs[0] + a_y * crs[1] + a_z * crs[2]
    
    return cell_matrix, inverse_matrix, unit_cell_volume
def get_atomic_coordinates(cifname):
    """
    ดึงข้อมูล atomic symbol และ Cartesian coordinates
    """
    ase_atoms = read(cifname)
    return ase_atoms.get_chemical_symbols(), ase_atoms.get_positions()
def RandomNumberOnUnitSphere():
	thetha = 0.0
	phi = 0.0
	theta = 2*PI*np.random.random_sample()
	phi = np.arccos(2*np.random.random_sample()-1.0)
	x = np.cos(theta)*np.sin(phi)
	y = np.sin(theta)*np.sin(phi)
	z = np.cos(phi)
	
	return x,y,z
def _dot_product(unit_cell_matrix, atom_positions):
    s_x = unit_cell_matrix.item(0) * atom_positions[0] + unit_cell_matrix.item(3) * atom_positions[1] + unit_cell_matrix.item(6) * atom_positions[2]
    s_y = unit_cell_matrix.item(1) * atom_positions[0] + unit_cell_matrix.item(4) * atom_positions[1] + unit_cell_matrix.item(7) * atom_positions[2]
    s_z = unit_cell_matrix.item(2) * atom_positions[0] + unit_cell_matrix.item(5) * atom_positions[1] + unit_cell_matrix.item(8) * atom_positions[2]
    new_coord = np.array([s_x, s_y, s_z], float)
    
    return new_coord
def ApplyBoundaryConditions(vec2, pos, cell_matrix, inverse_matrix):
	w = [0,0,0]
	x = [0,0,0]
	vec2 = copy.deepcopy(np.asarray(vec2))
	inverse_matrix2 = copy.deepcopy(np.asarray(inverse_matrix))
	fractional = _dot_product(inverse_matrix, vec2)
	# apply boundary conditions
	x[0] = fractional[0] - np.rint(fractional[0])
	x[1] = fractional[1] - np.rint(fractional[1])
	x[2] = fractional[2] - np.rint(fractional[2])
	cartesian = _dot_product(cell_matrix,x)
	
	return cartesian
def CheckSurfaceAreaOverlap(pAtom, pos, cell_matrix, atom_type, vdW_pAtom, fAtom_b, inverse_matrix):
	"""
	ตรวจสอบว่า Probe กับอะตอมซ้อนทับกันไหม
	"""
	well_depth_factor = 1.0
	fAtom_o = [0,0,0]
	distance = [0,0,0]
	# start enumerating all atoms in the object
	for i, elem in enumerate(pos):
		fAtom_o = copy.deepcopy(elem)
		if not np.array_equal(fAtom_o,fAtom_b):
			vdW_fAtom = vdw_radii.get(atom_type[i])
			equilibrium_distance = well_depth_factor * 0.5 * (vdW_pAtom + vdW_fAtom)
			fAtom_o = _dot_product(cell_matrix,fAtom_o)
			distance[0] = pAtom[0] - fAtom_o[0]
			distance[1] = pAtom[1] - fAtom_o[1]
			distance[2] = pAtom[2] - fAtom_o[2]
			dr = ApplyBoundaryConditions(distance, pos, cell_matrix, inverse_matrix)
			rr = (dr[0] * dr[0]) + (dr[1] * dr[1]) + (dr[2] * dr[2])
			if rr < (equilibrium_distance * equilibrium_distance):
				return True
				
	return False
def gsa(atom_type, pos, cell_matrix, inverse_matrix, unit_cell_volume, probe_diameter=1.0, nSample=20, equilibrium_distance=0.0, InsertTypeOfAtoms=None):
	
	"""
	Computes geometric surface area (GSA) of atoms. Return GSA in m^2/cm^3.

	atom_type : type of atom
	pos : coordinates
	cell_matrix : Crystall cell matrix (A, B, C)
	inverse_matrix: Inverse of crystall cell matrix
	probe_diameter : Diameter of a probe
	nSample : Number of sample probes
	equilibrium_distance = Equilibrium distance upon probe and atom in structure
	InsertTypeOfAtoms = Type of atom insert
	"""
	total = 0.0
	counted = 0.0
	vec = []
	examined_coordinates = []
	SurfaceAreaAverage = 0.0
	well_depth_factor = 1.0
	
	# start enumerating all atoms in the framework
	for i, elem in enumerate(atom_type):
		total=0.0
		counted=0.0
		if np.linalg.norm([pos[i][0], pos[i][1], pos[i][2]]) not in examined_coordinates:
			# check atomType to see if we want to insert probes around the atom
			if InsertTypeOfAtoms is atom_type[i]:
				vdW_fAtom = vdw_radii.get(elem)
				# Lorentz rule of mixing hard spheres 
				equilibrium_distance = well_depth_factor * (0.5 * (probe_diameter + vdW_fAtom))
				# start MC sampling for a given #
				for attempt in range(nSample):
					pAtom = np.array([0.0, 0.0, 0.0], float)
					total += 1
					# find a random number around the sphere
					vec = RandomNumberOnUnitSphere()
					# calculate the coordinates of the center of probe using vec and equilibrium distance
					pAtom[0] = pos[i][0]+vec[0]*equilibrium_distance
					pAtom[1] = pos[i][1]+vec[1]*equilibrium_distance
					pAtom[2] = pos[i][2]+vec[2]*equilibrium_distance
					# store the x,y,z coordinates of probe in an array
					vec2 = np.array([pAtom[0], pAtom[1], pAtom[2]])
					# check for the overlap between probe and the other framework atoms
					overlap = CheckSurfaceAreaOverlap(pAtom, pos, cell_matrix, atom_type, probe_diameter, pos[i], inverse_matrix)
					if not overlap:
						counted += 1

				# print("fraction of insertion near", atom_type[i], "is", counted/total)
				tmp = (counted/total)*4.0*PI*(equilibrium_distance * equilibrium_distance)

				SurfaceAreaAverage += tmp
				tmp2 = np.array([pos[i][0], pos[i][1], pos[i][2]])
				examined_coordinates.append(np.linalg.norm(tmp2))

	# print ("Surface Area in Ang^2: ", SurfaceAreaAverage)
	# print ("volume of unit cell: ", unit_cell_volume)
	# print ("Surface Area in m^2/cm^3: ", 1e4*SurfaceAreaAverage/unit_cell_volume)
	
	return 1e4*SurfaceAreaAverage/unit_cell_volume
def vsa(atom_type, pos, cell_matrix, inverse_matrix, unit_cell_volume, probe_diameter=1.0, nSample=20, total=0, equilibrium_distance=0.0):
	"""
	Computes the volumetric surface area (VSA) of atoms. Return VSA in m^2/cm3.

	atom_type : type of atom
	pos : coordinates
	cell_matrix : Crystall cell matrix (A, B, C)
	probe_diameter : Diameter of a probe
	nSample : Number of sample probes
	equilibrium_distance = Equilibrium distance upon probe and atom in structure
	"""
	counted = 0.0
	vec = np.zeros((1,3), float)
	SurfaceAreaAverage = 0.0
	well_depth_factor = 1.0
	fractional_list = []

	# start enumerating all atoms in the framework
	for i, elem in enumerate(atom_type):
		total = 0.0
		counted = 0.0
		vdW_fAtom = vdw_radii.get(elem)
		# Lorentz rule of mixing hard spheres 
		equilibrium_distance = well_depth_factor * 0.5 * (probe_diameter + vdW_fAtom)

		# start MC sampling for a given #
		for attempt in range(nSample):
			pAtom = np.array([0.0, 0.0, 0.0], float)
			total += 1
			# find a random number around the sphere
			vec = RandomNumberOnUnitSphere()
			# calculate the coordinates of the center of probe using vec and equilibrium distance
			fAtom_xyz = _dot_product(cell_matrix, pos[i])
			pAtom[0] = fAtom_xyz[0] + vec[0]*equilibrium_distance
			pAtom[1] = fAtom_xyz[1] + vec[1]*equilibrium_distance
			pAtom[2] = fAtom_xyz[2] + vec[2]*equilibrium_distance

			# check for the overlap between probe and the other framework atoms
			overlap = CheckSurfaceAreaOverlap(pAtom, pos, cell_matrix, atom_type, probe_diameter, pos[i], inverse_matrix)
			if not overlap:
				counted += 1

		tmp = (counted/total)*4.0*PI*(equilibrium_distance * equilibrium_distance)

		SurfaceAreaAverage += tmp
		fractional_list.append([elem[0], counted / total, tmp, 1.0e4*tmp/unit_cell_volume])
		# print(i, " out of ", len(atom_type))

	# fractional_list.sort(key=itemgetter(0))

	# file_out = open('freq.txt', 'w')
	# for index, items in groupby(fractional_list, itemgetter(0)):
	# 	sum_area = 0.0
	# 	for atom in items:
	# 		sum_area += atom[2]
	# 		file_out.write('\t'.join(str(i) for i in atom) + '\n')

	return 1.0e4*SurfaceAreaAverage/unit_cell_volume

def get_cif_file(number):
  return './mofsxml/mof_unit_'+str(number)+'.cif'

def surface_area(number, type, nSample):
  cifname = get_cif_file(number)
  cell_matrix, inverse_matrix, unit_cell_volume = get_box_matrix(cifname)
  atom_type, pos = get_atomic_coordinates(cifname)
  if type=="gsa":
    gsa_sum = 0
    for i in set(atom_type):
        gsa_ = gsa(atom_type, pos, cell_matrix, inverse_matrix, unit_cell_volume, probe_diameter=3.40, nSample=nSample, InsertTypeOfAtoms=i)
        gsa_sum += gsa_
    return gsa_sum / 1.66053
  if type=="vsa":
    return vsa(atom_type, pos, cell_matrix, inverse_matrix, unit_cell_volume, probe_diameter=3.40, nSample=nSample)

In [ ]:
surface_area(1, "gsa", 300)

/usr/local/lib/python3.7/dist-packages/ase/io/cif.py:404: UserWarning: crystal system 'triclinic' is not interpreted for space group 1. This may result in wrong setting!
  setting_name, spacegroup))


650.2219424809276

In [ ]:
surface_area(1, "gsa", 300)

/usr/local/lib/python3.7/dist-packages/ase/io/cif.py:404: UserWarning: crystal system 'triclinic' is not interpreted for space group 1. This may result in wrong setting!
  setting_name, spacegroup))


583.1553311893167

In [ ]:
surface_area(1, "gsa", 300)

/usr/local/lib/python3.7/dist-packages/ase/io/cif.py:404: UserWarning: crystal system 'triclinic' is not interpreted for space group 1. This may result in wrong setting!
  setting_name, spacegroup))


607.3872504232672

In [ ]:
surface_area(1, "gsa", 3000)

/usr/local/lib/python3.7/dist-packages/ase/io/cif.py:404: UserWarning: crystal system 'triclinic' is not interpreted for space group 1. This may result in wrong setting!
  setting_name, spacegroup))


594.1918288212856

In [ ]:
df[df['surface_area [m^2/g]']==0]

,MOFname,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,functional_groups,metal_linker,organic_linker1,organic_linker2,topology,CO2/N2_selectivity,heat_adsorption_CO2_P0.15bar_T298K [kcal/mol],CO2_working_capacity [mL/g]
0,mof_unit_1,1116.667429,875.240600,0.0,0.07899,53,3,4,11,5,22.864166,6.786041,105.284502
4,mof_unit_5,1137.800963,901.736120,0.0,0.07778,234,2,1,22,5,30.001838,6.478063,79.210001
11,mof_unit_12,1175.475860,1397.051280,0.0,0.05704,181,2,22,24,5,58.106974,7.407898,101.964812
13,mof_unit_14,1071.619874,701.628600,0.0,0.10455,262,2,12,22,5,31.664456,7.427183,133.371121
19,mof_unit_20,919.305781,689.174400,0.0,0.07816,20,3,10,18,5,47.332721,7.957782,136.387400
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68589,mof_unit_68590,2981.283155,2629.066240,0.0,0.00486,62,9,18,29,8,4.613801,NaN,-3.478823
68593,mof_unit_68594,1524.505449,1685.041936,0.0,0.00699,241,10,46,48,2,9.460791,NaN,-3.792634
68595,mof_unit_68596,3760.489888,3343.200672,0.0,0.01221,227,10,42,42,2,0.227160,NaN,-4.115523
68609,mof_unit_68610,1506.660363,1493.296496,0.0,0.01108,380,10,42,46,2,6.071818,NaN,-12.985582


In [ ]:
df[df['heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]'].isna()]

,MOFname,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,functional_groups,metal_linker,organic_linker1,organic_linker2,topology,CO2/N2_selectivity,heat_adsorption_CO2_P0.15bar_T298K [kcal/mol],CO2_working_capacity [mL/g]
66533,mof_unit_66534,708.220211,623.151240,0.0,0.01744,348,3,10,11,5,10.156757,NaN,12.718818
66539,mof_unit_66540,1134.838907,957.795860,0.0,0.02309,302,2,4,14,5,23.835969,NaN,6.162638
66544,mof_unit_66545,2526.333310,2372.883760,0.0,0.02510,254,9,18,18,8,17.749801,NaN,4.162900
66546,mof_unit_66547,2161.272866,2668.267736,0.0,0.00683,293,10,45,49,2,0.000000,NaN,8.119455
66549,mof_unit_66550,2372.870245,2885.422720,0.0,0.02459,13,3,22,22,5,6.204062,NaN,6.030742
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68581,mof_unit_68582,1108.951518,941.517200,0.0,0.01790,397,3,14,24,5,0.000000,NaN,-2.740601
68589,mof_unit_68590,2981.283155,2629.066240,0.0,0.00486,62,9,18,29,8,4.613801,NaN,-3.478823
68593,mof_unit_68594,1524.505449,1685.041936,0.0,0.00699,241,10,46,48,2,9.460791,NaN,-3.792634
68595,mof_unit_68596,3760.489888,3343.200672,0.0,0.01221,227,10,42,42,2,0.227160,NaN,-4.115523


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/k4kirin/mofsxml/main/train.csv',keep_default_na=True)
df[df.isna()]

,MOFname,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],functional_groups,metal_linker,organic_linker1,organic_linker2,topology,CO2/N2_selectivity,heat_adsorption_CO2_P0.15bar_T298K [kcal/mol],CO2_working_capacity [mL/g]
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
